# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') # импортируем библиотеку pandas и сохраняем датасет в переменной df
df # выводим на экран датасет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [2]:
df.info() # проверяем столбцы датасета для оформления выводов перед предобработкой


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

При первом взгляде на данные уже видно проблемы в столбцах:
* days_employed содержит минусовые значения, что возможно только если за точку отсчета брали какую либо определенную дату как в системе Unix, данные было бы возможно обработать, если бы мы знали дату в которую взяли кредит, есть так же и данные исчисляемые тысячей лет, что могло произойти если запятая находится не на своем месте, эти данные могут оказаться бесполезными ввиду невозможности запросить дополнительную информацию у разработчиков;
* в purpose одни и те же причины записаны по-разному. Пример: недвижимость, операции с жильем;
* total_income и days_employed включают пропуски. Возможно, эти столбцы являются не обязательными для заполнения.

## Шаг 2. Предобработка данных

### Обработка пропусков

Пустые строки столбца total_income составляют почти 11% от всех строк датасета, их удаление может сказаться на итоговом выводе, поставим вместо пропусков медиану дохода по каждому виду занятости. Так же запоним медианными значениями столбец days_employed, хотя возможно данные из него нам не пригодятся.

Сначала посчитаем медиану по каждому типу занятости

In [3]:
df_transformed = df.groupby('income_type')['total_income'].transform('median') # создадим датасет в котором соберем значения медианы по каждому типу занятости 
df['total_income'] = df['total_income'].fillna(df_transformed) # заменим пропуски на данный из датасета data_transformed
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) # замена значений days_employed
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** 

В датасете больше нет пустых строк.

### Замена типа данных
В столбце total_income находятся данные типа float, их лучше заменить на int, так как в таком формате с ними удобнее работать и читать.

In [4]:
df['total_income'] = df['total_income'].astype('int') # меняем тип данных на int методом astype, он подходит для этих целей. 
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

Теперь с данными в столбце total_income удобнее работать.

### Обработка дубликатов
Проверяем наличие явных дубликатов:

In [5]:
df.duplicated().sum() # подсчёт явных дубликатов

54

Убираем явные дубликаты:

In [6]:
df = df.drop_duplicates().reset_index(drop= True)  # удаление дубликатов методом drop_duplicates() и восстановление индексации строк
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


В столбце education есть слова начинающиеся с заглавной буквы, лучше проверить другие столбцы на схожие проблемы в данных.

In [7]:
 df['education'].unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [8]:
 df['family_status'].unique() 
    

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
df['income_type'].unique() 

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [10]:
df['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Похоже проблема характерная только для столбца education, скорее всего его заполняют вручную, заменим все буквы столбца на строчные:

In [11]:
df['education'] = df['education'].str.lower() # меняем регистр
df['education'].unique() # проверка значений столбца

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

Явных дубликатов в таблице больше нет, так же значения столбца education привели к единому стандарту.

### Лемматизация

В столбце purpose цель кредита судя по всему записывалась в свободной форме, но её нужно привести к определенным категориям. Сделаем лемматизацию столбца.

In [12]:
from pymystem3 import Mystem # импортируем из библиотеки pymystem3 инструмент Mystem
m = Mystem() 
def lemma(text): # пишем функцию которая преобразует данные столбца в леммы
    row = ' '.join(m.lemmatize(text))
    return row
df['purpose'] = df['purpose'].apply(lemma) # применяем фунцию к столбцу
df['purpose'].unique() # проверяем сработал ли лемматизация

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

**Вывод**

После лемматизации можно выделить четыре основных категории:
* Покупка недвижимости
* Автокредит
* Образование
* Свадьба

Есть категории, по которым лучше уточнить цель кредита, к примеру, "строительство   жилой   недвижимость". Данная категория может быть воспринята как покупка стройматериалов или нецелевой кредит и как покупка недвижимости в "долевке"(Покупка недвижимости), так как нельзя уточнить информацию у заказчика отнесем её к Покупке недвижимости, анологичным способом распределим другие сомнительные варианты. 

### Категоризация данных
Лемматизация произведена, можно приступить к разделению данных на категории.

In [13]:
def category(row): # создаем функцию, которая приведет данные в единый формат.
    if 'жилье' in row:
        return 'покупка недвижимости'
    if 'недвижимость' in row:
        return 'покупка недвижимости'
    if 'автомобиль' in row:
        return 'автокредит'
    if 'свадьба' in row:
        return 'свадьба'
    if 'образование' in row:
        return 'образование'
df['purpose'] = df['purpose'].apply(category) # применяем функцию к столбцу purpose
df['purpose'].unique() # проверяем все ли значения столбца были изменены

array(['покупка недвижимости', 'автокредит', 'образование', 'свадьба'],
      dtype=object)

**Вывод**

Данные приведены в более удобный для чтения вид, можно приступить к анализу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос проверим какие значения столбца есть в датасете.

In [14]:
df['children'].unique() # смотрим значения столбца для разделения на группы

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [15]:
df[df['children'] == -1].count() # подсчитаем количество сомнительных значений

children            47
days_employed       47
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        47
purpose             47
dtype: int64

В столбце есть значения -1, нельзя сказать, что они значит, так как оно может означать как ошибку так и потерю ребенка. Уточнить не получится, поэтому оставим данные как есть и будем учитывать вместе со значением ноль, тем более их кол-во составляет всего 0.2% и веса в итоге они не будут иметь. Напишем функцию, которая пройдется по столбцам children и debt и разделит данные:

In [16]:
def debt_children(raw): # создаем функцию для группировки данных
    children = raw['children']
    debt = raw['debt']
    if children <= 0:
        if debt == 1:
            return 'без детей, долг'
    if children <= 0:
        if debt == 0:
            return 'без детей, нет долга' 
    if children > 0:
        if debt == 0:
            return 'есть  дети, нет долга'
    if 0 < children > 0:
        if debt == 1:
            return 'есть  дети, долг'
        
df['addiction_children'] = df.apply(debt_children, axis = 1) # применяем функцию к столбцу

def child(raw): # создаем функцию для группировки данных для сводной таблицы
    children = raw['children']
    if children <= 0:
        return 'нет детей'
    if children > 0:
        return 'есть дети' 
df['child'] = df.apply(child, axis = 1)   

df.pivot_table(index=['child'], columns='addiction_children', values='total_income', aggfunc='count') # выводим на экран сводную таблицу

addiction_children,"без детей, долг","без детей, нет долга","есть дети, долг","есть дети, нет долга"
child,,,,
есть дети,NaN,NaN,677.0,6640.0
нет детей,1064.0,13090.0,NaN,NaN


Так как количественно группы не равны лучше будет перевести количество должников в каждой в проценты и сравнить:

In [17]:
no_childe_debt = 1064/14154*100 # процент 'без детей, долг'
childe_debt = 677/7317*100 # процент 'среднее кол-во детей, долг'

display('без детей, долг:', no_childe_debt, 'есть  дети, долг:', childe_debt)


'без детей, долг:'

7.51730959446093

'есть  дети, долг:'

9.25242585759191

**Вывод**

В итоге мы видим, что есть зависимость между долгом и детьми у заемщика, возможно это связано с тратами на содержание детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогично предыдущему вопросу проведем анализ:

In [18]:
def debt_family(raw): # создаем функцию для группировки данных
    family = raw['family_status']
    debt = raw['debt']
    if family == 'женат / замужем':
        if debt == 1:
            return 'в браке, долг'
    if family == 'женат / замужем':
        if debt == 0:
            return 'в браке, нет долга' 
    if family == 'гражданский брак':
        if debt == 1:
            return 'не в браке, долг'
    if family == 'гражданский брак':
        if debt == 0:
            return 'не в браке, нет долга'
    if family == 'вдовец / вдова':
        if debt == 1:
            return 'вдовец, долг'
    if family == 'вдовец / вдова':
        if debt == 0:
            return 'вдовец, нет долга'
    if family == 'в разводе':
        if debt == 1:
            return 'в разводе, долг'
    if family == 'в разводе':
        if debt == 0:
            return 'в разводе, нет долга' 
    if family == 'Не женат / не замужем':
        if debt == 1:
            return 'холост, долг'
    if family == 'Не женат / не замужем':
        if debt == 0:
            return 'холост, нет долга'    
df['addiction_family'] = df.apply(debt_family, axis = 1) # применяем функцию к столбцу
df.pivot_table(index=['family_status'], columns='addiction_family', values='total_income', aggfunc='count') # выводим на экран сводную таблицу

addiction_family,"в браке, долг","в браке, нет долга","в разводе, долг","в разводе, нет долга","вдовец, долг","вдовец, нет долга","не в браке, долг","не в браке, нет долга","холост, долг","холост, нет долга"
family_status,,,,,,,,,,
Не женат / не замужем,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274.0,2536.0
в разводе,NaN,NaN,85.0,1110.0,NaN,NaN,NaN,NaN,NaN,NaN
вдовец / вдова,NaN,NaN,NaN,NaN,63.0,896.0,NaN,NaN,NaN,NaN
гражданский брак,NaN,NaN,NaN,NaN,NaN,NaN,388.0,3775.0,NaN,NaN
женат / замужем,931.0,11413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
merried_debt = 931/12344*100 # процент 'в браке, долг'
unmerried_debt = 388/4163*100 # процент 'не в браке, долг'
single_debt = 274/2810*100 # процент 'холост, долг'
divorced_debt = 85/1195*100 # процент 'в разводе, долг'
widow_debt = 63/959*100 # процент 'вдовец, долг'
display(['в браке, долг:', merried_debt, 'не в браке, долг:', unmerried_debt, 'холост, долг:', single_debt, 'в разводе, долг:', divorced_debt, 'вдовец, долг:', widow_debt] )

['в браке, долг:',
 7.542125729099157,
 'не в браке, долг:',
 9.320201777564256,
 'холост, долг:',
 9.750889679715302,
 'в разводе, долг:',
 7.112970711297072,
 'вдовец, долг:',
 6.569343065693431]

**Вывод**

Судя по данным чаще всего имеют долги люди находящиеся в гражданском браке и холостые, в остальных группах процент должников ниже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Так как у нас нет данных от клиента о том как сгруппировать доходы и нам неизвестна страна в которой брали срез, чтобы посмотреть статистические данные в интернете, рассчитаем группы математическим методом, возьмем значение медианы и разделиме её на два, чтобы получить шаг которым мы разделим доход на четыре четверти, таким образом мы получим четыре группы: низкий доход, доход ниже среднего, доход выше среднего, высокий доход.

In [20]:
df['total_income'].median()/2 # считаем медиану столбца total_income и делим 

71297.0

In [21]:
def debt_income(raw): # создаем функцию для группировки данных
    income = raw['total_income']
    debt = raw['debt']
    if income < 71297:
        if debt == 1:
            return 'низкий доход, долг'
    if income < 71297:
        if debt == 0:
            return 'низкий доход, нет долга' 
    if 71297 < income < 145017:
        if debt == 1:
            return 'доход ниже среднего, долг'
    if 71297 < income < 142594:
        if debt == 0:
            return 'доход ниже среднего, нет долга'
    if 142594 < income < 213891:
        if debt == 1:
            return 'доход выше среднего, долг'
    if 142594 < income < 213891:
        if debt == 0:
            return 'доход выше среднего, нет долга' 
    if 213891 < income:
        if debt == 1:
            return 'высокий доход, долг'
    if 213891 < income:
        if debt == 0:
            return 'высокий доход, нет долга'     
df['addiction_income'] = df.apply(debt_income, axis = 1) # применяем функцию к столбцу
df.pivot_table(index=['debt'], columns='addiction_income', values='total_income', aggfunc='count') # выводим на экран сводную таблицу

addiction_income,"высокий доход, долг","высокий доход, нет долга","доход выше среднего, долг","доход выше среднего, нет долга","доход ниже среднего, долг","доход ниже среднего, нет долга","низкий доход, долг","низкий доход, нет долга"
debt,,,,,,,,
0,NaN,4016.0,NaN,5766.0,NaN,7499.0,NaN,1470.0
1,304.0,NaN,500.0,NaN,828.0,NaN,109.0,NaN


In [22]:
low_salary_debt = 109/1579*100 # процент 'низкий доход, долг'
lower_middle_debt = 828/8327*100 # процент 'доход ниже среднего, долг'
upper_middle_debt = 500/6266*100 # процент 'доход выше среднего, долг' 
high_salary_debt = 304/4320*100 # процент 'высокий доход, долг'

display('низкий доход, долг:', low_salary_debt, 'доход ниже среднего, долг:', lower_middle_debt, 'доход выше среднего, долг:', upper_middle_debt, 'высокий доход, долг:', high_salary_debt)


'низкий доход, долг:'

6.903103229892336

'доход ниже среднего, долг:'

9.943557103398582

'доход выше среднего, долг:'

7.979572294924991

'высокий доход, долг:'

7.037037037037037

**Вывод**

Из выше указанного анализа видно, что люди с высоким, выше среднего и низким доходом более ответственно относятся к выплате кредита, возможно это связанно с тем что люди с низким доходом более тщательно планируют свои расходы, а люди с высоким и доходом выше среднего имеют достаточно свободных средств для закрытия кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
def debt_purpose(raw): # создаем функцию для группировки данных
    purpose = raw['purpose']
    debt = raw['debt']
    if purpose == 'покупка недвижимости':
        if debt == 1:
            return 'недвижимость, долг'
    if purpose == 'покупка недвижимости':
        if debt == 0:
            return 'недвижимость, нет долга' 
    if purpose == 'автокредит':
        if debt == 1:
            return 'автокредит, долг'
    if purpose == 'автокредит':
        if debt == 0:
            return 'автокредит, нет долга'
    if purpose == 'образование':
        if debt == 1:
            return 'образование, долг'
    if purpose == 'образование':
        if debt == 0:
            return 'образование, нет долга'
    if purpose == 'свадьба':
        if debt == 1:
            return 'свадьба, долг'
    if purpose == 'свадьба':
        if debt == 0:
            return 'свадьба, нет долга' 
    
df['addiction_purpose'] = df.apply(debt_purpose, axis = 1) # применяем функцию к столбцу

df.pivot_table(index=['purpose'], columns='addiction_purpose', values='total_income', aggfunc='count') # выводим на экран сводную таблицу

addiction_purpose,"автокредит, долг","автокредит, нет долга","недвижимость, долг","недвижимость, нет долга","образование, долг","образование, нет долга","свадьба, долг","свадьба, нет долга"
purpose,,,,,,,,
автокредит,403.0,3905.0,NaN,NaN,NaN,NaN,NaN,NaN
образование,NaN,NaN,NaN,NaN,370.0,3644.0,NaN,NaN
покупка недвижимости,NaN,NaN,782.0,10032.0,NaN,NaN,NaN,NaN
свадьба,NaN,NaN,NaN,NaN,NaN,NaN,186.0,2149.0


In [24]:
real_estate_debt = 782/10814*100 # процент 'недвижимость, долг'
auto_debt = 403/4308*100 # процент 'автокредит, долг'
education_debt = 370/4014*100 # процент 'образование, долг'
marriage_debt = 186/2335*100 # процент 'свадьба, долг'
display(['недвижимость, долг:', real_estate_debt, 'автокредит, долг:', auto_debt, 'образование, долг:', education_debt, 'свадьба, долг:', marriage_debt])

['недвижимость, долг:',
 7.231366746809691,
 'автокредит, долг:',
 9.35468895078923,
 'образование, долг:',
 9.217737917289487,
 'свадьба, долг:',
 7.965738758029979]

**Вывод**

Чаще всего заемщики имеют долги по автокредитам и образованию. В случае с автокредитом это может быть связано с тем, что транспортное средство можно легко повредить или довести до полной гибели, в следствии этого заемщик может либо не захотеть платить, либо у него может не хватить денег. Образование долгосрочный процесс и возможно долги связаны с тем, что не всегда после окончания учебы получается устроиться на работу.

## Шаг 4. Общий вывод

По вышеуказанным данным рекомендую учесть в кредитном скорринге, что заемщики с детьми, а также холостые или находящиеся в гражданском браке чаще имею долги по кредиту. Так же помимо заявленных заказчиком параметров предлагаю учесть в скорринге цель кредита: автокредит, образование, так как в этих категориях чаще всего имеют долги по возврату кредита, помимо этого для скоринга предлагаю рассмотреть другой рассчет ставки кредита для клиентов с доходами ниже среднего, ввиду более высокого количества долгов при возврате средств.